In [2]:

from flask import Flask, render_template, Response, jsonify
import cv2
import numpy as np
from tensorflow.keras.models import load_model

app = Flask(__name__)

# Load the trained model
MODEL_PATH = "convlstm_model.h5"
model = load_model(MODEL_PATH)

# Define class labels
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Z']

# Webcam capture
camera = cv2.VideoCapture(0)

IMG_SIZE = 64

def preprocess_frame(frame):
    # Resize the frame to match the input size of the model
    resized_frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    # Normalize the frame
    normalized_frame = resized_frame / 255.0
    # Expand dimensions to match ConvLSTM input: (1, 1, IMG_SIZE, IMG_SIZE, 3)
    return np.expand_dims(np.expand_dims(normalized_frame, axis=0), axis=1)

def generate_frames():
    while True:
        success, frame = camera.read()
        if not success:
            break

        # Mirror the frame for better UX
        frame = cv2.flip(frame, 1)

        # Draw a rectangle for the ROI
        roi_start = (200, 100)
        roi_end = (400, 300)
        cv2.rectangle(frame, roi_start, roi_end, (0, 255, 0), 2)

        # Extract the ROI
        roi = frame[roi_start[1]:roi_end[1], roi_start[0]:roi_end[0]]

        # Preprocess the ROI for prediction
        try:
            preprocessed_frame = preprocess_frame(roi)
            prediction = model.predict(preprocessed_frame)
            predicted_label = class_labels[np.argmax(prediction)]
        except Exception as e:
            predicted_label = "Error"

        # Display the prediction on the frame
        cv2.putText(frame, f"Prediction: {predicted_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Encode the frame as a byte stream
        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Apr/2025 14:29:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Apr/2025 14:29:44] "GET /static/style.css HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [22/Apr/2025 14:29:53] "GET /video_feed HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step


INFO:werkzeug:127.0.0.1 - - [22/Apr/2025 14:29:53] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 